# __NEW ROADMAP__

https://github.com/openai/spinningup/blob/master/spinup/algos/tf1/trpo/core.py

*   TRPO implemented using diagonal multivariate normal distributions (looking at two sources and some info in OG paper)
*   gradient is calculated as usual, it will be remapped via inverse Hessian
*   objective function contains KL div constraint, which is a Hessian of KL divergence.
*   It can be computed fast and ez with conjugate gradient (CG). But we need to calculate KL div. Unlike using Fisher Information Matrix (FIM)

STEPS

0.  Run the experiment (this is somewhere inside point 1.)
1.  Define MLP Gaussian policy
    *   it takes in list of states and list of actions, which were conducted based on states using old version of policy
    *   MLP produces mean value parameter for each action thats possible in a given state.

        so if state list is of shape (steps, num_observables), MLP will produce parameters of shape (steps, num_actions). only mean. stdev is unique for each action

    *   we sample actions using reparametrization trick: 
        $$a_\pi \sim \mu(states) + \N(\theta)*\sigma$$
        are these logits? im not sure. should be same shape as \mu-> (steps, num_actions)

    *   compute log likelihood of $a_\pi$, sum over actions dim (might be multiplication in log representation)

    *   compute log likelihood for observed actions $a$. This implies that dims of $a$ and $a_\pi$ are the same so no distribution over actions? Not possible, since mu has last dim of num_actions.

    *   retrieve $\mu$ and $\sigma$ of old policy. What to do if num states were different??

    *   compute analytic KL divergence using old and new distribution parameters. 



2.  Compute Surrogate reward
    *   ratio of log likelihoods new/old
    *   times advantage -> objective f-n
3.  Update weights
    *   get MLP params
    *   calculate grad w.r.t. objective f-n
    *   define hessian-vector product function for arbitrary vector
    *   perform CG iterations to get direction and initial step size
    *   backtracking line search

4. update value function (baseline)


# Surrogate reward $(U(\theta))$
(https://www.youtube.com/watch?v=KjWF8VIMGiY)

We can define a loss which, when optimized, will produce an incremental improvement to a policy with respect to old policy version.

Given we have gathered trajectory using old policy, we can evaluate expected rewards using new policy via _Importance Sampling_.

My explanation can be seen midway throughout [Sutton_P1_Ch05_Monte_Carlo.ipynb notebook (off-policy)](../../RL_Reinforced_Learning/Sutton_P1_Ch05_Monte_Carlo.ipynb)

$$U(\theta) = \mathbb{E}_{\tau \sim \theta_{old}} \bigg[ \frac{P(\tau|\theta)}{P(\tau|\theta_{old})} R(\tau)\bigg]$$
Gradient of this expected return is:
$$\nabla_{\theta}U(\theta) = \mathbb{E}_{\tau \sim \theta_{old}} \bigg[ \frac{\nabla_{\theta} P(\tau|\theta)}{P(\tau|\theta_{old})} R(\tau)\bigg]$$
if old policy and new policy are the same ($\theta = \theta_{old}$), via
$$\nabla_{\theta}P(\tau|\theta) = P(\tau|\theta)\cdot\nabla_{\theta} \ log P(\tau|\theta)$$
we recover 'vanilla' policy gradient update ([Simplest_Policy_Gradient](01_Simplest_Policy_Gradient_Implementations.ipynb)):
$$\underset{\tau \sim \pi}{\mathbb{E}}\bigg[ \sum_{t=0}^{T} \nabla_{\theta} \ log  \ \pi_\theta(a_t|s_t) \cdot R(\tau) \bigg]$$

One may say that case $\theta = \theta_{old}$ is a first (linear) term in approximation of $\nabla_{\theta}U(\theta)$

One caveat is that rewards $R(\tau)$ are gathered under policy and if new policy changes drastically, it invalidates reward.

Goal is to reduce difference of old and new policies, so rewards can be reused.

This can be done by enforcing $\epsilon$ - small KL divergence $D_{KL}(\pi||\pi_{old}) \leq \epsilon$

$$D_{KL}(P||Q) \ \dot = \ \sum_{x \in X} P(x) \cdot \ log \ \bigg( \frac{P(x)}{Q(x)}\bigg)$$

Similarly to case of MC importance sampling, many product terms in ratio $\frac{P(\tau|\theta)}{P(\tau|\theta_{old})}$ will cancel out, due to being the same for current environment.

In fact expression reduces to
$$D_{KL}\bigg( P(\tau|\theta) \ || \ P(\tau|\theta_{old})\bigg) = \sum_\tau P(\tau|\theta) \cdot \frac{\prod_{t = 0}^{T} \pi_\theta(a_t|s_t)}{\prod_{t = 0}^{T} \pi_{\theta_{old}}(a_t|s_t)}$$

<style>
img {
  display: block;
  margin-left: auto;
  margin-right: auto;
}
</style>

<img  src="https://spinningup.openai.com/en/latest/_images/math/5808864ea60ebc3702704717d9f4c3773c90540d.svg" 
      width=460 
      style="background-color:white"/>

step 8. Start unpacking from the end:
1. KL divergence of sample

conjugate gradient method explored in

[Notes_Method_of_Conjugate_Gradient_Descent.ipynb](../../optimization/Notes_Method_of_Conjugate_Gradient_Descent.ipynb)







step 9. uses backtracking line search. notes in

[Notes_Line_search.ipynb](../../optimization/Notes_Line_search.ipynb)